In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

In [2]:
#//----------------PRECARGA-------------------------//
#//Ruta de data // Base Data (Without Deleting) 
segmentos_fulll = get_fresh_query_result("https://internal-redash.federate.frubana.com/",101609,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
city_code = 'BOG'
    #year-month-day
todays_date = '2022-07-22' 
non_buyers_date = '2022-06-22' #1 months
customer_date_desde = '2022-04-22' #3 months
    #data for sku sales data
sku_date_desde = '2022-05-22' #2 months

t_cambio = 3000 #COL=3000 // MX=19.19 // BR=3.88

### Loading COSTUMERS INFO

In [5]:
#//--------------------------------------
#//Ruta de data // Base Data (Without Deleting)
if city_code in ('BAQ', 'BOG', 'CMX', 'SPO'):
    segmentos_full = segmentos_fulll[segmentos_fulll.city == city_code].copy()
else:
    data = [[city_code,'NA',0]]
    segmentos_full = pd.DataFrame(data, columns=['city', 'microsegment', 'customer_id'])

In [6]:
#//--------------------------------------
#//Ruta de data // KAM's costumers to delete
#//KAMs customers me lo manda Emiliano Lomeli (SE ACTUALIZAN DE UN SCRIPT MES A MES)
users_eliminar_kam = pd.read_csv("import_data/segments/KAMs_JUN.csv")['costumer_id'].unique().tolist()

In [7]:
#//--------------------------------------
#//Ruta de data // Base Data (Delete previous week - month loop)
# Guardado como NB_MS_semana_anterior_{city_code} o en query https://redash.federate.frubana.com/queries/82561 para inicio de mes "segmentos_full"
#segmentos_prev1 = pd.read_csv(f'import_data/segments/NB_MS_semana_anterior_1_{city_code}.csv') 
#segmentos_prev2 = pd.read_csv(f'import_data/segments/NB_MS_semana_anterior_2_{city_code}.csv') 
#segmentos_prev3 = pd.read_csv(f'import_data/segments/NB_MS_semana_anterior_3_{city_code}.csv') 

In [8]:
#//Users to ignore  ->>> ACAA [(ACA IMPORTANTE VER EL SEGMENTOS_PREV2 QUE ESTAS TRAYENDO)]
#//At the start of the month we just delete KAM users, then Users that already got SD
users_eliminar = users_eliminar_kam #+ segmentos_prev1['customer_id'].unique().tolist() #+ segmentos_prev2['customer_id'].unique().tolist() + segmentos_prev3['customer_id'].unique().tolist()

In [9]:
#//Must do the same length
print(len(users_eliminar_kam))
#print(len(segmentos_prev1['customer_id'].unique().tolist()))
#print(len(segmentos_prev2['customer_id'].unique().tolist()))
#print(len(segmentos_prev3['customer_id'].unique().tolist()))
len(users_eliminar)

2492


2492

### Loading FORCED_SKUS // Cleaning SEGMENT_NAMES

In [10]:
#//--------------------------------------
#//Forced skus
#if city_code in test_list):
if city_code in ('BAQ', 'BOG', 'CMX', 'SPO'):
    forced_skus = pd.read_csv(f'import_data/skus/{city_code}_forced_skus.csv')
    forced_skus['microsegment'] = forced_skus['microsegment'].apply(lambda x: x.replace("'",''))
else:
    data = [['NA','NA','NA',0]]
    forced_skus = pd.DataFrame(data, columns=['microsegment', 'subcat', 'name', 'sku_ids'])

In [11]:
def prepareSku_ids(skus): #ESTO SE HACE POR LOS HIJOS!
    skus = str(skus)
    if ', ' in skus:
        skus  = skus.replace(', ','#')        
    return skus
#To clean forced_skus DATA
forced_skus['sku_ids'] = forced_skus['sku_ids'].apply(prepareSku_ids)  

#To fix names if they get an error
if city_code in ('BAQ', 'BOG', 'CMX', 'SPO'):
    forced_skus['microsegment'] = forced_skus['microsegment'].apply(lambda x: x.replace('Empanadas/Arepas','Empanadas/arepas'))
    forced_skus['microsegment'] = forced_skus['microsegment'].apply(lambda x: x.replace('Parrilla/Asadero','Parrilla/asadero'))

In [12]:
forced_skus['microsegment'].unique()

array(['Corrientazo', 'Panadería/Pastelería', 'Comida rápida',
       'Empanadas/arepas', 'Parrilla/asadero', 'Comida especializada',
       'Cafetería'], dtype=object)

In [13]:
segmentos_full['microsegment'].unique()

array(['Comida rápida', 'Empanadas/Arepas', 'Comida especializada',
       'Panadería/Pastelería', 'Corrientazo c/desayuno',
       'Corrientazo s/desayuno', 'Parrilla/Asadero', 'Cafetería',
       'Panadería/Pastelería/Cafetería', 'Fritos'], dtype=object)

In [14]:
#//Clean segment_names
segmentos_full['microsegment'] = segmentos_full['microsegment'].str.replace('s/café da manhã','')
segmentos_full['microsegment'] = segmentos_full['microsegment'].str.replace('c/café da manhã','')
segmentos_full['microsegment'] = segmentos_full['microsegment'].str.replace('c/desayuno','')
segmentos_full['microsegment'] = segmentos_full['microsegment'].str.replace('s/desayuno','')
segmentos_full['microsegment'] = segmentos_full['microsegment'].str.strip()
segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: 'Taquería' if 'Taquería' in x else x)
segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: x.replace('Parrilla/Asadero','Parrilla/asadero'))

if city_code == 'BOG':
#BOG
    segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: x.replace('Empanadas/Arepas','Empanadas/arepas'))
    segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: x.replace('Fritos','Empanadas/arepas'))
    segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: 'Panadería/Pastelería' if 'Panadería/Pastelería/Cafetería' in x else x)
elif city_code == 'BAQ':
#BAQ
    segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: 'Panadería/Pastelería/Cafetería' if 'Cafetería' in x else x)
    segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: 'Panadería/Pastelería/Cafetería' if 'Panadería/Pastelería' in x else x)
    segmentos_full['microsegment'] = segmentos_full['microsegment'].apply(lambda x: x.replace('Empanadas/Arepas','Fritos'))

forced_skus['microsegment'] = forced_skus['microsegment'].str.strip()
forced_skus['microsegment'] = forced_skus['microsegment'].str.replace('Comida saludable/Juguerías','Juguería/Comida saludable')

In [15]:
segmentos_full['microsegment'].unique()

array(['Comida rápida', 'Empanadas/arepas', 'Comida especializada',
       'Panadería/Pastelería', 'Corrientazo', 'Parrilla/asadero',
       'Cafetería'], dtype=object)

In [16]:
forced_skus['microsegment'].unique()

array(['Corrientazo', 'Panadería/Pastelería', 'Comida rápida',
       'Empanadas/arepas', 'Parrilla/asadero', 'Comida especializada',
       'Cafetería'], dtype=object)

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [17]:
#//Remove skus 
#It is used because some SKUs can get in trouble with supplier for the SD or to exclude them for another activity 
skus_remove = [
166212,
283894,
290295,
166210,
165914,
175066,
165916,
94250,
231534,
105657,
175054,
165866,
290296,
283896,
175068,
198505,
165899,
232256,
165909,
283895,
232355,
165922,
165924,
165930,
232350,
165932,
179696,
256705,
256693,
173947,
53047,
179697,
-301275,
274988,
312799,
192151,
-300546,
256511,
274987,
192150,
256512,
-301275,
192147,
192148,
256694,
173946,
168100,
168101,
168100,
232345,
165938,
165940,
165907,
232365,
165946,
166222,
441216,
165901,
232257,
165948,
198505,
165868,
198495,
232370,
175056,
232339,
165882,
165884,
441216,
441217,
166054,
166056,
232360, #HASTA ACA
262884,
262885,
262886,
443352,
443351,
271395,
271396,
271392,
99017,
324145,
324143,
56606,
56612,
-301818, #Diana
124661, #Diana
124659, #Diana
56612, #alquería
401540, #alquería 
56606, #alquería
90176, #alquería
266755, #alquería
126505, #alquería
#### experimento mayo
-303030,
124708,
124710,
-303057,
124702,
124700,
124716,
63697,
280865,
280868,
280862,
440367,
-304254,
-302913,
277542,
-300543,
62912,
212438,
-302130,
210161,
210159,
97137,
83969,
241989,
271594,
271602,
147339,
254113,
254112,
31507,
82996,
102355,
111139,
118025,
118027,
118020,
-304413,
48733,
-302271,
-302286,
367833,
136080,
136082,
130690,
290511,
169189,
290224,
290223,
290222,
377870,
377869,
377868,
219702,
37211,
219697,
37188,
296040,
296041,
91360,
91356,
106890,
106889,
106895,
162844,
162845,
162843,
278268,
322342,
278267,
285339,
285338,
195864,
195961,
195960,
290048,
290049,
290047,
218839,
218840,
218841,
197104,
197105,
359613,
196789,
196790,
196791,
250239,
250240,
197116,
342968,
342969,
342967,
197114,
197113,
197718,
197719,
197720,
195993,
195992,
195994,
304504,
249435,
249433,
249434,
290283,
371569,
371567,
371571,
371573,
254974,
254842,
254971,
254844,
254865,
254856,
295085,
254973,
336632,
249954,
249953,
249993,
249992,
404804,
404799,
285549,
285348,
285347,
296190,
285353,
285548,
285552,
285551,
285354,
285345,
332389,
296188,
296189,
285344,
332390,
126139,
491240,
191539,
1207,
490568,
1209,
1265,
123039,
491238,
333292,
333294,
333296,
333298,
489261,
333300,
333301,
323989,
323990,
323995,
323996,
323998,
323999,
322945,
322946,
489263,
366572,
366573,
366575,
271393,
367096,
367101,
491189,
489257,
490566,
147341,
489259,
489253,
490567,
367142,
491188,
120813,
491239
]

In [18]:
def detectSKUs(skus, strs):
    
    if  str(strs)=='nan':
        return True #To exclude nans    
    for sku in skus:
        if str(sku) in strs:
            return True
    return False

In [19]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [20]:
#Quick fix of a category error
data_ventas['category'] = data_ventas['category'].apply(lambda x: 'Bebidas' if 'Cervezas y Licores' in x else x)

In [21]:
data_ventas.shape

(1014435, 19)

In [22]:
data_ventas = data_ventas[~data_ventas['sku_id'].isin(skus_remove)]
forced_skus = forced_skus[~forced_skus['sku_ids'].apply(lambda x: detectSKUs(skus_remove, x))]

In [23]:
#//customers for offers */(has further uses)*/
microsegmentacion_df = segmentos_full[(~segmentos_full['customer_id'].isin(users_eliminar))]
data_ventas = data_ventas[(~data_ventas['customer_id'].isin(users_eliminar))]

### Create the MS "Other" for BAU 

In [24]:
data_full = data_ventas.merge(microsegmentacion_df[['customer_id','microsegment']], how = 'left', on = 'customer_id')

In [25]:
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [26]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique()

,customer_id
microsegment,
Cafetería,1248
Comida especializada,1100
Comida rápida,2033
Corrientazo,5559
Empanadas/arepas,1451
Other,3693
Panadería/Pastelería,4192
Parrilla/asadero,1330


In [27]:
#Get the same variable to do the process job
data_ventas = data_full

In [28]:
#//customers for offers, has NB and BAU
microsegments_users = segmentos_full[(~segmentos_full['customer_id'].isin(users_eliminar)) & (segmentos_full['city'] == city_code)].customer_id.unique()
no_microsegments_users = data_ventas[~data_ventas['customer_id'].isin(users_eliminar)].customer_id.unique()

In [29]:
ids_costumers = pd.Series(list(set(np.concatenate((microsegments_users,no_microsegments_users)))), name='customer_id')

In [30]:
##### Importante tener claro y no sobreescribir ######

#ids_costumers.to_csv(f"import_data/segments/NB_MS_semana_anterior_1_{city_code}.csv", index=False)
#ids_costumers.to_csv(f"import_data/segments/NB_MS_semana_anterior_2_{city_code}.csv", index=False)
#ids_costumers.to_csv(f"import_data/segments/NB_MS_semana_anterior_3_{city_code}.csv", index=False)

#->>> ACAA SE CREA ESTE ARCHIVO PARA NEXT WEEK

### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [31]:
#Current Segments for City (Not used at the moment) // 'Other' is BAU
Segment_type= {'BOG':['Comida rápida', 'Panadería/Pastelería', 'Corrientazo',
       'Cafetería', 'Parrilla/asadero', 'Empanadas/arepas',
       'Comida especializada', 'Other'],
               
              'BAQ': ['Parrilla/asadero', 'Corrientazo', 'Tienda/Restaurante',
       'Comida de mar y Colombiana', 'Panadería/Pastelería/Cafetería',
       'Comida especializada', 'Comida rápida', 'Fritos'],
               
              'CMX':['Fonda', 'Juguería/Comida saludable', 'Comida rápida', 'Taquería',
       'Cafetería', 'Comida especializada', 'Panadería/Pastelería', 'Other'],
               
              'SPO':['Lanches', 'Comida internacional/Grill',
                       'Comida brasileira y saudavel', 'Prato do dia',
                       'Pizza/Comida Italiana', 'Sucos/Açaí', 'Padaria/Cafeteria/Bolos',
                       'Comida por quilo', 'Other'],
              'MDE':['Other'],
              'GDL':['Other'],
              'CWB':['Other'],
              'BHZ':['Other'],
              'VCP':['Other'],
              'PBC':['Other']
              }

In [38]:
# parametros = {
#     "BOG":{
#         "Corrientazo":['Gaseosas'], 
        
#         'Panadería/Pastelería':['Gaseosas'], 
        
#         'Comida rápida':['Gaseosas'],
        
#         'Empanadas/arepas':['Gaseosas'], 
        
#         'Parrilla/asadero':['Gaseosas'], 
                                    
#         'Comida especializada':['Gaseosas'], 
        
#         'Cafetería':['Gaseosas'], 
        
#         'Other':['Gaseosas']
#     }
# }

In [37]:
parametros = {
    "BAQ":{
        "Corrientazo":['Pollo','Cerdo','Res','Granos','Huevos','Verduras','Frutas','Arroz','Gaseosas','Contenedores','Vasos','Bolsas',
                       'Tubérculos','Detergente, Jabón & Lavalozas','Papel & Toallas','Desinfectantes & Sanitizantes',
                       'Implementos de Aseo','Pescados & Mariscos Congelado','Aceites & Grasas','Cubiertos & Pitillos',
                       'Azúcar & Endulzantes'],
        
        'Comida de mar y Colombiana':['Pescados & Mariscos Congelado','Arroz','Tubérculos','Frutas','Verduras','Aceites & Grasas',
                                      'Leches','Huevos','Gaseosas','Res','Contenedores','Bolsas','Vasos','Granos','Cerdo',
                                      'Detergente, Jabón & Lavalozas','Desinfectantes & Sanitizantes'],
        
        "Comida rápida":['Aceites & Grasas','Embutidos','Pollo','Alimentos Congelados','Quesos','Contenedores','Cubiertos & Pitillos',
                         'Papel & Toallas','Salsas','Tubérculos','Verduras','Gaseosas','Vasos','Bolsas','Res','Platos & Bandejas',
                         'Detergente, Jabón & Lavalozas','Desinfectantes & Sanitizantes','Cerdo','Harinas & Mezclas',
                         'Frutas','Huevos','Arroz','Leches','Implementos de Aseo','Granos','Cremas'],
        
        "Fritos":['Aceites & Grasas','Harinas & Mezclas','Huevos','Pollo','Frutas','Verduras','Tubérculos','Papel & Toallas','Leches',
                  'Detergente, Jabón & Lavalozas','Platos & Bandejas','Bolsas','Gaseosas','Res','Embutidos','Vasos',
                  'Arroz','Desinfectantes & Sanitizantes'],
        
        "Panadería/Pastelería/Cafetería":['Panadería & Repostería','Azúcar & Endulzantes','Aceites & Grasas','Huevos','Leches','Pollo',
                                          'Quesos','Gaseosas','Bolsas','Vasos','Cubiertos & Pitillos','Papel & Toallas',
                                          'Derivados Lácteos','Verduras','Cremas','Platos & Bandejas','Café, Chocolate e Infusiones',
                                          'Frutas','Res','Arroz','Harinas & Mezclas','Embutidos','Otros Abarrotes',
                                          'Tubérculos','Granos','Implementos de Aseo','Pasta'], 
        
        'Parrilla/asadero':['Pollo','Cerdo','Res','Pescados & Mariscos Congelado','Embutidos','Alimentos Congelados','Verduras',
                            'Tubérculos','Bolsas','Aceites & Grasas','Arroz','Salsas','Gaseosas','Contenedores',
                            'Detergente, Jabón & Lavalozas','Implementos de Aseo','Cremas','Pasta','Jugos',
                            'Sal & Sazonadores'],
        
        'Comida especializada':['Pescados & Mariscos Congelado','Res','Verduras','Salsas','Arroz','Contenedores','Papel & Toallas',
                                'Quesos','Embutidos','Pasta','Gaseosas','Tubérculos','Pollo','Vasos','Bolsas','Huevos',
                                'Detergente, Jabón & Lavalozas','Frutas','Aceites & Grasas',
                                'Desinfectantes & Sanitizantes'],
        
        'Tienda/Restaurante':['Gaseosas','Papel & Toallas','Huevos','Aceites & Grasas','Cubiertos & Pitillos','Verduras',
                              'Tubérculos','Pollo','Arroz','Granos','Frutas','Leches','Embutidos','Vasos',
                              'Platos & Bandejas','Pescados & Mariscos Congelado','Harinas & Mezclas','Bolsas',
                              'Café, Chocolate e Infusiones','Pasta','Salsas','Cremas'],
        #,'Quesos'
        'Other':['Pollo','Res','Cerdo','Pescados & Mariscos Congelado','Embutidos','Tubérculos','Arroz','Frutas','Huevos','Verduras',
                 'Vasos','Contenedores','Detergente, Jabón & Lavalozas','Gaseosas','Alimentos Congelados','Papel & Toallas',
                 'Desinfectantes & Sanitizantes','Cubiertos & Pitillos','Leches','Implementos de Aseo','Empaque & Envoltura','Cremas',
                 'Bolsas','Quesos','Platos & Bandejas','Complementos','Derivados Lácteos','Mantequillas & Margarinas',
                 'Salsas','Granos','Pasta','Enlatados','Panadería & Repostería','Café, Chocolate e Infusiones']       
    },
    
    "BOG":{
        "Corrientazo":['Pollo','Cerdo','Res','Granos','Huevos','Verduras','Frutas Jugo','Leches','Arroz','Vasos','Bolsas','Contenedores',
                       'Gaseosas','Tubérculos','Detergente, Jabón & Lavalozas','Papel & Toallas','Desinfectantes & Sanitizantes',
                       'Pescados & Mariscos Congelado','Cubiertos & Pitillos','Implementos de Aseo','Salsas','Pasta','Embutidos',
                       'Platos & Bandejas','Cervezas & Licores','Empaque & Envoltura'], 
        
        'Panadería/Pastelería':['Panadería & Repostería','Azúcar & Endulzantes','Harinas & Mezclas','Arroz','Aceites & Grasas',
                                'Huevos','Pollo','Bolsas','Cubiertos & Pitillos','Papel & Toallas','Leches','Derivados Lácteos',
                                'Mantequillas','Cremas','Quesos','Café, Chocolate e Infusiones','Frutas','Frutas Jugo','Enlatados',
                                'Res','Embutidos','Gaseosas','Platos & Bandejas','Vasos','Detergente, Jabón & Lavalozas',
                                'Desinfectantes & Sanitizantes','Implementos de Aseo'],
        
        'Comida rápida':['Aceites & Grasas','Embutidos','Pollo','Res','Alimentos Congelados','Quesos','Harinas & Mezclas','Salsas',
                         'Verduras','Tubérculos','Gaseosas','Contenedores','Platos & Bandejas','Bolsas','Cubiertos & Pitillos',
                         'Vasos','Detergente, Jabón & Lavalozas','Papel & Toallas','Desinfectantes & Sanitizantes','Frutas',
                         'Implementos de Aseo','Leches','Granos','Pasta','Empaque & Envoltura'],
        
        'Empanadas/arepas':['Aceites & Grasas','Huevos','Bolsas','Cubiertos & Pitillos','Pollo','Quesos','Papel & Toallas',
                            'Harinas & Mezclas','Tubérculos','Café, Chocolate e Infusiones','Embutidos','Gaseosas','Platos & Bandejas',
                            'Vasos','Detergente, Jabón & Lavalozas','Desinfectantes & Sanitizantes','Leches',
                            'Implementos de Aseo','Contenedores','Salsas','Sal & Sazonadores','Granos'],
        
        'Parrilla/asadero':['Pollo','Res','Cerdo','Pescados & Mariscos Congelado','Embutidos','Alimentos Congelados','Arroz',
                            'Verduras','Tubérculos','Bolsas','Salsas','Contenedores','Detergente, Jabón & Lavalozas',
                            'Papel & Toallas','Aceites & Grasas','Desinfectantes & Sanitizantes','Gaseosas',
                            'Leches','Pasta','Frutas Jugo',
                            'Cervezas & Licores','Platos & Bandejas'],
                                    
        'Comida especializada':['Pescados & Mariscos Congelado','Res','Verduras','Salsas','Arroz','Papel & Toallas','Quesos',
                                'Embutidos','Pasta','Tubérculos','Huevos','Gaseosas','Pollo','Contenedores','Vasos','Bolsas',
                                'Detergente, Jabón & Lavalozas','Desinfectantes & Sanitizantes','Cubiertos & Pitillos','Leches',
                                'Implementos de Aseo','Embutidos','Frutas Jugo','Cremas',
                                'Café, Chocolate e Infusiones','Aguas','Enlatados','Empaque & Envoltura',
                                'Platos & Bandejas','Derivados Lácteos'],
        
        'Cafetería':['Harinas & Mezclas','Azúcar & Endulzantes','Huevos','Frutas','Frutas Jugo','Verduras','Pollo','Papel & Toallas',
                     'Café, Chocolate e Infusiones','Derivados Lácteos','Mantequillas','Quesos','Embutidos','Gaseosas','Contenedores',
                     'Vasos','Bolsas','Cubiertos & Pitillos','Detergente, Jabón & Lavalozas','Desinfectantes & Sanitizantes','Cremas',
                     'Leches','Tubérculos','Granos','Implementos de Aseo',
                     'Pasta','Platos & Bandejas','Panadería & Repostería'],
        
        'Other':['Pollo','Res','Cerdo','Pescados & Mariscos Congelado','Embutidos','Tubérculos','Verduras','Frutas','Huevos',
                 'Aceites & Grasas','Vasos','Contenedores','Papel & Toallas','Arroz','Frutas Jugo','Gaseosas','Cubiertos & Pitillos',
                 'Bolsas','Alimentos Congelados','Leches','Detergente, Jabón & Lavalozas','Complementos','Quesos',
                 'Implementos de Aseo','Desinfectantes & Sanitizantes','Cremas','Empaque & Envoltura','Platos & Bandejas',
                 'Mantequillas','Derivados Lácteos']
    },
    
    'CMX':{
        'Fonda':['Derivados Lácteos','Pollo','Cerdo','Res','Arroz','Pasta','Leches','Quesos','Verduras','Huevos','Papel & Toallas',
                 'Salsas','Enlatados','Frutas','Tubérculos','Chiles Secos','Aceites & Grasas','Embutidos',
                 'Panadería & Tortilleria','Pescados & Mariscos Congelado','Alimentos Congelados','Café, Chocolate e Infusiones',
                 'Mantequillas & Margarinas','Empaque & Envoltura'],
        
        'Taquería':['Cerdo','Res','Pescados & Mariscos Congelado','Quesos','Verduras','Frutas','Enlatados','Granos',
                    'Panadería & Tortilleria','Papel & Toallas','Vasos, Contenedores & Platos','Cubiertos','Pollo','Embutidos',
                    'Aceites & Grasas','Chiles Secos','Detergente, Jabón & Lavatrastes','Sal & Sazonadores','Arroz','Pasta',
                    'Salsas','Cremas','Harinas & Mezclas','Jugos','Huevos'], 
        
        'Juguería/Comida saludable':['Frutas','Verduras','Granos','Leches','Detergente, Jabón & Lavatrastes',
                                     'Vasos, Contenedores & Platos','Papel & Toallas','Pollo','Cubiertos','Quesos',
                                     'Enlatados','Derivados Lácteos','Implementos de Aseo','Desinfectantes & Sanitizantes','Jugos',
                                     'Alimentos Congelados','Bolsas'],
        
        'Panadería/Pastelería':['Azúcar & Endulzantes','Harinas & Mezclas','Aceites & Grasas','Leches','Huevos',
                                'Vasos, Contenedores & Platos','Bolsas','Cubiertos','Papel & Toallas','Derivados Lácteos','Quesos',
                                'Mantequillas & Margarinas','Café, Chocolate e Infusiones','Embutidos','Frutas',
                                'Mermeladas y Confituras','Cremas','Empaque & Envoltura','Cuidado & Protección Personal','Salsas'], 
        
        'Comida rápida':['Aceites & Grasas','Embutidos','Alimentos Congelados','Quesos','Salsas','Verduras','Panadería & Tortilleria',
                         'Detergente, Jabón & Lavatrastes','Tubérculos','Papel & Toallas','Pollo','Res','Vasos, Contenedores & Platos',
                         'Empaque & Envoltura','Cubiertos','Cerdo','Harinas & Mezclas','Derivados Lácteos','Enlatados','Pasta','Granos',
                         'Chiles Secos','Arroz'], 
        
        'Comida especializada':['Pescados & Mariscos Congelado','Res','Verduras','Arroz','Salsas','Detergente, Jabón & Lavatrastes',
                                'Vasos, Contenedores & Platos','Aceites & Grasas','Derivados Lácteos','Papel & Toallas','Pasta',
                                'Alimentos Congelados','Pollo','Embutidos','Cerdo','Huevos',
                                'Desinfectantes & Sanitizantes','Harinas & Mezclas','Granos','Sal & Sazonadores',
                                'Chiles Secos'],              
      
        'Cafetería':['Azúcar & Endulzantes','Quesos','Leches','Frutas','Embutidos','Papel & Toallas','Verduras',
                     'Vasos, Contenedores & Platos','Derivados Lácteos','Salsas','Café, Chocolate e Infusiones','Cubiertos',
                     'Mermeladas y Confituras','Pollo','Huevos','Mantequillas & Margarinas','Cremas','Bolsas',
                     'Detergente, Jabón & Lavatrastes','Granos','Enlatados','Tubérculos','Aceites & Grasas'],
        
        'Other':['Pollo','Res','Cerdo','Pescados & Mariscos Congelado','Embutidos','Azúcar & Endulzantes','Tubérculos','Granos','Frutas',
                 'Verduras','Huevos','Leches','Vasos, Contenedores & Platos','Detergente, Jabón & Lavatrastes','Arroz','Pasta','Quesos',
                 'Derivados Lácteos','Papel & Toallas','Empaque & Envoltura','Mantequillas & Margarinas','Implementos de Aseo',
                 'Desinfectantes & Sanitizantes','Cremas','Alimentos Congelados','Cubiertos','Bolsas','Cuidado & Protección Personal']                      
    },
    
    'SPO':{
        
        'Prato do dia': ['Arroz','Feijão','Tubérculos','Carne Vermelha','Embutidos','Frutas','Farinhas e Misturas','Cervejas',
                         'Refrigerantes e Energéticos','Sucos e Chás','Águas e Isotônicos','Verduras','Legumes','Frango','Suínos',
                         'Alimentos Congelados','Azeites, Óleos e Vinagres','Detergente, Sabão e Lava-Louças','Manteigas e Margarinas',
                         'Batatas Pré-Fritas'],
        
        'Comida por quilo' : ['Arroz','Farinhas e Misturas','Frutas','Verduras','Legumes','Hortaliças','Tubérculos','Feijão',
                              'Carne Vermelha','Embutidos','Frango','Alimentos Congelados','Massas e Molhos',
                              'Detergente, Sabão e Lava-Louças','Refrigerantes e Energéticos','Cervejas','Sucos e Chás',
                              'Águas e Isotônicos','Suínos','Manteigas e Margarinas','Azeites, Óleos e Vinagres','Batatas Pré-Fritas',
                              'Embalagens','Ovos','Utensílios','Polpas e Açaí','Sacos','Desinfetantes e Sanitizantes',
                              'Desengordurantes','Peixes e Frutos do Mar','Grãos'],
        
        'Lanches':['Tubérculos','Alimentos Congelados','Verduras','Legumes','Azeites, Óleos e Vinagres','Embutidos',
                   'Batatas Pré-Fritas','Carne Vermelha','Suínos','Queijos','Cervejas','Refrigerantes e Energéticos','Sucos e Chás',
                   'Águas e Isotônicos','Ovos','Papéis','Frango','Condimentos','Utensílios','Detergente, Sabão e Lava-Louças',
                   'Embalagens','Sacos','Polpas e Açaí','Desinfetantes e Sanitizantes','Desengordurantes','Copos, Recipientes e Pratos',
                   'Peixes e Frutos do Mar'],                            

        'Padaria/Cafeteria/Bolos':['Farinhas e Misturas','Azeites, Óleos e Vinagres','Açúcar, Adoçantes e Doces','Leite','Ovos',
                                   'Queijos','Manteigas e Margarinas','Laticínios','Frango','Embutidos','Embalagens',
                                   'Detergente, Sabão e Lava-Louças','Cervejas','Refrigerantes e Energéticos','Sucos e Chás',
                                   'Águas e Isotônicos','Papéis','Café, Chocolates e Infusões','Frutas','Utensílios','Batatas Pré-Fritas',
                                   'Doces e Sobremesas','Sacos'],
        
        'Comida brasileira y saudavel': ['Verduras','Legumes','Tubérculos','Carne Vermelha','Detergente, Sabão e Lava-Louças',
                                         'Peixes e Frutos do Mar','Farinhas e Misturas','Cervejas','Batatas Pré-Fritas',
                                         'Refrigerantes e Energéticos','Águas e Isotônicos','Cervejas','Leite','Frutas','Feijão',
                                         'Temperos','Queijos','Alimentos Congelados','Embalagens','Sucos e Chás','Ovos','Polpas e Açaí',
                                         'Papéis','Outros Limpadores','Embutidos','Condimentos','Café, Chocolates e Infusões','Grãos'],
        
        'Comida internacional/Grill': ['Peixes e Frutos do Mar','Frango','Carne Vermelha','Suínos','Legumes','Cervejas','Tubérculos',
                                       'Alimentos Congelados','Embutidos','Queijos','Arroz','Condimentos','Frutas','Verduras',
                                       'Sucos e Chás','Refrigerantes e Energéticos','Águas e Isotônicos',
                                       'Detergente, Sabão e Lava-Louças','Batatas Pré-Fritas','Temperos','Outros Limpadores','Feijão',
                                       'Ovos','Desinfetantes e Sanitizantes','Utensílios','Desengordurantes','Sacos','Polpas e Açaí'],
        
        'Pizza/Comida Italiana':['Queijos','Leite','Farinhas e Misturas','Massas e Molhos','Temperos','Verduras','Conservas e Enlatados',
                                 'Legumes','Embutidos','Suínos','Cervejas','Refrigerantes e Energéticos','Sucos e Chás','Arroz',
                                 'Laticínios','Azeites, Óleos e Vinagres','Batatas Pré-Fritas','Águas e Isotônicos','Embalagens',
                                 'Detergente, Sabão e Lava-Louças','Ovos'],
        
        'Sucos/Açaí':['Frutas','Polpas e Açaí','Açúcar, Adoçantes e Doces','Papéis','Tubérculos','Batatas Pré-Fritas',
                      'Águas e Isotônicos','Refrigerantes e Energéticos','Sucos e Chás','Utensílios','Legumes','Embalagens','Sacos',
                      'Frango','Detergente, Sabão e Lava-Louças','Outros Limpadores','Laticínios','Temperos','Condimentos','Embutidos',
                      'Arroz','Feijão','Café, Chocolates e Infusões'],
        
        'Other':['Frango','Suínos','Embutidos','Carne Vermelha','Peixes e Frutos do Mar','Tubérculos','Verduras','Frutas','Legumes',
                 'Leite','Ovos','Detergente, Sabão e Lava-Louças','Feijão','Farinhas e Misturas','Cervejas','Refrigerantes e Energéticos',
                 'Sucos e Chás','Águas e Isotônicos','Batatas Pré-Fritas','Sacos','Desengordurantes','Grãos',
                 'Copos, Recipientes e Pratos','Embalagens','Limpadores de Piso','Doces e Sobremesas']
    },
    
    'MDE':{
        'Other':['Tubérculos','Arroz','Frutas','Huevos','Verduras','Vasos','Contenedores','Detergente, Jabón & Lavalozas',
                 'Gaseosas','Papel & Toallas','Desinfectantes & Sanitizantes','Cubiertos & Pitillos','Leches','Empaque & Envoltura',
                 'Cremas','Bolsas','Platos & Bandejas','Derivados Lácteos','Cervezas & Licores','Sal & Sazonadores',
                 'Enlatados','Panadería & Repostería']
    },
    
    'GDL':{
        'Other':['Pollo','Res','Cerdo','Pescados & Mariscos','Embutidos','Azúcar & Endulzantes','Tubérculos','Granos','Frutas',
                 'Verduras','Huevos','Leches','Vasos, Contenedores & Platos','Detergente, Jabón & Lavatrastes','Arroz','Pasta',
                 'Quesos','Derivados Lácteos','Papel & Toallas','Mantequillas','Implementos de Aseo','Cremas','Alimentos Congelados',
                 'Cubiertos','Bolsas','Sal & Sazonadores','Chiles Secos','Mermeladas y Confituras','Empaque & Envoltura',
                 'Desinfectantes & Sanitizantes']
    },
    
    'PBC':{
        'Other':['Pollo', 'Res', 'Cerdo','Pescados & Mariscos','Embutidos',"Azúcar & Endulzantes", "Tubérculos", "Granos", "Frutas", "Verduras", "Huevos","Leches",
                 "Vasos, Contenedores & Platos","Detergente, Jabón & Lavatrastes","Arroz",'Pasta','Quesos', 'Derivados Lácteos', 'Papel & Toallas',
                 'Empaque & Envoltura','Implementos de Aseo', 'Detergente, Jabón & Lavatrastes', 'Cremas',
                 'Alimentos Congelados', 'Cubiertos', 'Bolsas','Empaque & Envoltura']
    },

    'CWB':{
        'Other':['Frango','Suínos','Embutidos','Carne Vermelha','Peixes e Frutos do Mar','Tubérculos','Verduras','Frutas',
                 'Legumes','Leite','Ovos','Detergente, Sabão e Lava-Louças','Feijão','Farinhas e Misturas','Cervejas',
                 'Refrigerantes e Energéticos','Sucos e Chás','Águas e Isotônicos','Arroz','Batatas Pré-Fritas',
                 'Café, Chocolates e Infusões','Condimentos','Hortaliças','Queijos','Manteigas e Margarinas',
                 'Conservas e Enlatados','Desinfetantes e Sanitizantes','Limpadores de Piso']
    },

    'BHZ':{
        'Other':['Frango','Suínos','Embutidos','Carne Vermelha','Peixes e Frutos do Mar','Tubérculos','Verduras','Frutas',
                 'Legumes','Leite','Ovos','Detergente, Sabão e Lava-Louças','Feijão','Farinhas e Misturas','Cervejas',
                 'Refrigerantes e Energéticos','Sucos e Chás','Utensílios de Limpeza','Açúcar, Adoçantes e Doces',
                 'Laticínios','Temperos','Desinfetantes e Sanitizantes','Hortaliças','Desengordurantes','Limpadores de Piso',
                 'Copos, Recipientes e Pratos','Grãos','Doces e Sobremesas']
    },
    
    'VCP':{
        'Other':['Tubérculos','Verduras','Frutas','Legumes','Leite','Ovos','Detergente, Sabão e Lava-Louças','Feijão','Farinhas e Misturas','Cervejas',
                 'Refrigerantes e Energéticos','Sucos e Chás','Águas e Isotônicos','Arroz','Café, Chocolates e Infusões','Condimentos',
                 'Conservas e Enlatados','Desinfetantes e Sanitizantes']
    }#'Frango','Suínos','Embutidos','Carne Vermelha','Peixes e Frutos do Mar','Batatas Pré-Fritas','Hortaliças','Queijos','Manteigas e Margarinas','Limpadores de Piso'
}

In [39]:
def normalize_text(raw_text):
    """
    Removes common accent characters and normalize text.
    """
    raw_text = raw_text.replace(" ", "")
    raw_text = raw_text.replace("�", "")
    raw_text = raw_text.replace("/", "")
    raw_text = re.sub(r'\s*', '', raw_text)
    raw_text = re.sub(r"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(r"[èéêë]", 'e', raw_text)
    raw_text = re.sub(r"[ìíîï]", 'i', raw_text) 
    raw_text = re.sub(r"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(r"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(r"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(r"[ß]", 'ss', raw_text)
    raw_text = re.sub(r"[ñ]", 'n', raw_text)
    raw_text = re.sub(r"[ç]", 'c', raw_text)
    raw_text = raw_text.lower()
    return raw_text

In [40]:
microsegmentacion_df[(microsegmentacion_df['city'] == city_code)]['microsegment'].value_counts()

Corrientazo             13257
Panadería/Pastelería     6942
Comida rápida            4380
Empanadas/arepas         3385
Cafetería                2471
Parrilla/asadero         2418
Comida especializada     2327
Name: microsegment, dtype: int64

In [41]:
#Sacar de aca los microsegmentos
if city_code in ('BAQ', 'BOG', 'CMX', 'SPO'):
    msg = microsegmentacion_df[(microsegmentacion_df['city'] == city_code)]['microsegment'].unique()
    msg = np.append(msg,'Other')
else:
    msg = np.array(['Other'])    
print(msg)

['Comida rápida' 'Empanadas/arepas' 'Comida especializada'
 'Panadería/Pastelería' 'Corrientazo' 'Parrilla/asadero' 'Cafetería'
 'Other']


### END

In [51]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_offer_csv_plt = pd.DataFrame()
global_segment_list = [] #//for procesing

In [52]:
#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    if microsegment != 'Other':
        #//Forced sku per microsegment
        forced_skus_formated = forced_skus[forced_skus['microsegment']==microsegment].groupby('subcat')['sku_ids'].unique().to_dict()
        #//Customers per microsegment
        customers_ms = microsegmentacion_df[(microsegmentacion_df['microsegment'] == microsegment) & (microsegmentacion_df['city'] == city_code)]['customer_id'].unique()
        #//Customers to give them the skus
        customers_for_skus_ms =  segmentos_full[(segmentos_full['microsegment'] == microsegment) & (segmentos_full['city'] == city_code)]['customer_id'].unique()
    else:
        forced_skus_formated = {}
        customers_ms = data_ventas[['microsegment','customer_id']][data_ventas.microsegment == 'Other']['customer_id'].unique()
        customers_for_skus_ms = data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == 'Other') & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()

    ms_name = normalize_text(microsegment) #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    #// Separation of types of sales (non_buyers/skus for offers)
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customers_ms)].copy()
    print(data_buyers.customer_id.nunique())
    
    #->>> ACAA
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=parametros[city_code][microsegment], level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)
    
    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        lst.append(d)
        
    #//appending to global list    
        global_segment_list.append(d)

    print(f'Done {city_code} {ms_name}')
#//End of big loop and start of data bundle    

//-------------------------------------------------------------------------------
Comida rápida
2033
Uso de data externa
Aceites & Grasas skus added succesfully
Alimentos Congelados skus added succesfully
ERROR WITH:  Bebidas Alcohólicas
Subcategory not recognized & removed from list
ERROR WITH:  Bebidas sin alcohol
Subcategory not recognized & removed from list
Bolsas skus added succesfully
Cerdo skus added succesfully
Contenedores skus added succesfully
Cubiertos & Pitillos skus added succesfully
ERROR WITH:  Cuidado & Protección Personal
Subcategory not recognized & removed from list
Desinfectantes & Sanitizantes skus added succesfully
Detergente, Jabón & Lavalozas skus added succesfully
Embutidos skus added succesfully
Harinas & Mezclas skus added succesfully
Implementos de Aseo skus added succesfully
Platos & Bandejas skus added succesfully
Pollo skus added succesfully
Quesos skus added succesfully
Res skus added succesfully
Salsas skus added succesfully
ERROR WITH:  Uniformes
Sub

In [53]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'export_data/MS/csv_{city_code}_All_MS.csv', index = False)

with open(f'export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'Done {city_code}')

Done BOG


### To get the segments ID (Should be after loading the segments in POSTMAN and FRIDA

In [ ]:
#global_offer_csv

In [ ]:
#offers_ids_df = get_fresh_query_result("https://internal-redash.federate.frubana.com/",88683,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

In [ ]:
#get_done_csv = global_offer_csv.merge(offers_ids_df[['offer_name','customer_segment_id_rdsh']], how = 'left', on = 'offer_name')

In [ ]:
#get_done_csv.customer_segment_id = get_done_csv.customer_segment_id_rdsh
#get_done_csv = get_done_csv.drop(columns = ['customer_segment_id_rdsh'])
#print(f'Done {city_code}')

In [ ]:
#get_done_csv